**This notebook contains a script that sends learnerships contracts to students**

**Add Local Library to Path**

In [165]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    sys.path.append(module_path + '/local_library')

**Import Libraries**

In [166]:
import pandas as pd
import requests
import json
import re
from datetime import date
from local_library import import_worksheet
from local_library import export_worksheet
from local_library import send_bulk_signrequest

**Create Library**

In [167]:
#split a column into multiple columns
def split_column_num(target_df, column, column_len):
    
    #create list from 0 to column_len
    n_list = list(range(0, column_len))
    
    #create and populate columns
    for i in n_list:
        target_df[f'{column}_{i+1}'] = [elem[i] if i < len(elem) else '' for elem in target_df[column]]
        
    return


#split column into two binary columns
def column_to_binary(target_df, column, specify='no'):
    
    #convert column to string
    target_df[column] = target_df[column].astype(str)
    
    #strip whitespace
    target_df[column] = target_df[column].str.strip()
    
    #create and populate binary columns
    target_df[f'{column}_yes'] = ['X' if elem.lower() == 'yes' else '' for elem in target_df[column]]
    target_df[f'{column}_no'] = ['X' if elem.lower() == 'no' else '' for elem in target_df[column]]
    
    return

#convert phone numbers
def convert_phone(target_df, column):
    target_df[column] = target_df[column].astype(str)
    target_df[column] = target_df[column].astype(float)
    target_df[column] = target_df[column].astype(int)
    target_df[column] = target_df[column].astype(str)
    
    return


#convert column type to string
def convert_columns_type_string(target_df, columns):
    for column in columns:
        target_df[column] = target_df[column].astype(str)

    return


#create age columns
def create_age_column(target_df, column):
    
    #convert dob to datetime formatt
    target_df[column] = pd.to_datetime(target_df[column])

    #set the date
    today = date.today()

    #calculate age
    target_df['Age'] = today.year - target_df[column].dt.year

    #convert age to int
    target_df['Age'] = target_df['Age'].astype(int)

    #mark if above 35 years
    target_df['above_35'] = ['X' if age > 35 else '' for age in target_df['Age']]

    #mark if below 36 years
    target_df['below_35'] = ['X' if age < 36 else '' for age in target_df['Age']]
    
    return


#split columns based on column values
def split_column(target_df, column):
    new_columns = target_df[column].unique()
    for new_column in new_columns:
        target_df[new_column] = ['X' if elem == new_column else '' for elem in target_df[column]]
        
    return


#split column based on status
def split_column_status(target_df, column, value):
    #mark if citizen yes
    target_df[f"{column}_yes"] = ['X' if elem == value else '' for elem in target_df[column]]

    #mark if citizen no
    target_df[f"{column}_no"] = ['X' if elem != value else '' for elem in target_df[column]]

    #specify citizenshp status
    target_df[f"{column}_specify"] = [elem if elem != value else '' for elem in target_df[column]]
    
    return


def split_disability(target_df, column, value):
    target_df[f"{column}_yes"] = ['X' if elem.lower() != value.lower() else '' for elem in target_df[column]]

    target_df[f"{column}_no"] = ['X' if elem.lower() == value.lower() else '' for elem in target_df[column]]

    target_df[f"{column}_specify"] = [elem if elem.lower() != value.lower() else '' for elem in target_df[column]]
    
    return


#add static column
def add_static_columns(target_df, column_dict):
    for key in column_dict.keys():
        target_df[key] = column_dict[key]
        
    return


#subset a dataframe
def subset_dataframe(target_df, column, subset_array):
    return target_df[target_df[column].isin(subset_array)]

**Import WorkSheets**

In [168]:
#import sponsorship data
cohort_df = import_worksheet("Cohort 2021 MICT System Onloading", "Sheet1")

#import tag ids
tag_ids_df = import_worksheet("Tag IDs","Student")

**Update Cohort Column Names**

In [169]:

column_names = {"Home Language":"home_language","Home Street":"home_address_1", "Home Suburb":"home_address_2", "Home City":"home_address_3",
                "Home Postal Code":"home_address_post_code", "Residential Street":"residential_address_1", "Residential Suburb":"residential_address_2",
                "Residential City":"residential_address_3", "Residential Postal Code":"residential_address_post_code", "Disability Type":"disability",
                "Date Of Birth":"birth_date", "First Name":"learner_name", "Last Name": "learner_surname",
                "Mobile Phone":"learner_phone", "Email Address":"learner_email",
                "Name of High School":"high_school", "Completion Year":"high_school_last_year",
                "Residential Status":"citizen", "ID / Passport Number":"id", "Gender":"gender", "Ethnicity":'ethnicity',
                "Home Province":"province", "Nationality":"country", "Highest Qualification":"highest_qualification",
                "Next Of Kin Name":"contact_name", "Next Of Kin Surname":"contact_surname", "Next of Kin Contact":"contact_phone",
                "Middle Name(s)":"middle_name"
               }

cohort_df = cohort_df.rename(columns=column_names)


**Convert Columns to Type String**

In [170]:
column_names = ['birth_date']

convert_columns_type_string(cohort_df, column_names)

**Convert Phone Number and High School Year**

In [171]:
#cnvert phone to type string
#convert_phone(cohort_df, 'learner_phone')
cohort_df['learner_phone'] = cohort_df['learner_phone'].astype(str)
if "." in cohort_df['learner_phone'].values[0]:
    cohort_df['learner_phone'] = [phone.split(".")[0] for phone in cohort_df['learner_phone']]


cohort_df['high_school_last_year'] = cohort_df['high_school_last_year'].astype(str)
if "." in cohort_df['high_school_last_year'].values[0]:
    cohort_df['high_school_last_year'] = [year.split(".")[0] for year in cohort_df['high_school_last_year']]

**Create ID column**

In [172]:
#use split_column
split_column_num(cohort_df, "id", 13)

**Filter empty columns**

In [176]:
#cohort_df = cohort_df[cohort_df['learner_phone'].notna()]
cohort_df = cohort_df[cohort_df['birth_date'].notna()]
#cohort_df = cohort_df[cohort_df['high_school_last_year'].notna()]


**Create Age Column**

In [177]:
#create age column
create_age_column(cohort_df, "birth_date")
cohort_df['birth_date'] = cohort_df['birth_date'].astype(str)

/Users/assistants/Library/Python/3.8/lib/python/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


**Create Next of kin Contact**

In [178]:
cohort_df['contact'] = cohort_df['contact_name'] + " " + cohort_df['contact_surname']

**Create Municipality**

In [179]:
cohort_df['municipality'] = cohort_df['province'].map({'Western Cape':'Cape Town Metro', 'Gauteng':'Johannesburg Metro',
       'Limpopo':'Blouberg','KwaZulu-Natal':'UMlazi municipality', 'North West':'Molopo',
       'Free State':'Maluti a Phofung', 'Eastern Cape':'Chris Hani District Municipality', 'Mpumalanga':'Emalahleni Local Municipality'})


**Create Gender Columns**

In [180]:
#strip gender column whitespaces
cohort_df['gender'] = cohort_df['gender'].str.strip()

#convert equity column to lowercase
cohort_df['gender'] = cohort_df['gender'].str.lower()

split_column(cohort_df, 'gender')

**Create Ethnicity Columns**

In [181]:
#strip equity column whitespaces
cohort_df['Equity'] = cohort_df['ethnicity'].str.strip()

#convert equity column to lowercase
cohort_df['Equity'] = cohort_df['Equity'].str.lower()

#reformat equity column
cohort_df['Equity'] = cohort_df['Equity'].replace({'black': 'african', 'coloured': 'coloured',
                             'indian': 'indian', 'asian': 'african',
                             'chinese': 'african', 'white': 'white'})

split_column(cohort_df, 'Equity')

**Create Highest Qualification**

In [182]:
#strip equity column whitespaces
cohort_df['highest_qualification'] = cohort_df['highest_qualification'].str.strip()

#convert equity column to lowercase
cohort_df['highest_qualification'] = cohort_df['highest_qualification'].str.lower()

#reformat equity column
cohort_df['highest_qualification'] = cohort_df['highest_qualification'].replace({'Higher Certificate': 'certificate',
                                    'below matric':'matric', 'bachelors degree':'degree', 'honours degree': 'degree',
                                    'masters degree':'degree', 'national certificate':'certificate', 'advanced diploma':'diploma',
                                    'higher certificate':'certificate'})

split_column(cohort_df, 'highest_qualification')

**Create Disability Column**

In [183]:
#strip disability column whitespaces
cohort_df['disability'] = cohort_df['disability'].str.strip()

#convert disability column to lowercase
cohort_df['disability'] = cohort_df['disability'].str.lower()

#reformat disability column
#cohort_df['disability'] = cohort_df['disability'].replace({'no': 'None', 'nan': 'None',
#                             'yes': 'Disabled but unspecified', 'yes - specwearer': 'None',
#                             'n': 'None', '-': 'None', 'visual impairment - spec wearer': 'None',
#                             'aspergers / autistic': 'Emotional (behav/psych)', 'patella alta':'Physical (move/stand etc)',
#                             'y- specwearer': 'None', 'add': 'None', 'visual impairment - spec wearer -': 'None',
#                             '\\n': 'None', 'y - specwearer':'None'})

#create disability yes, no, status columns
split_disability(cohort_df, 'disability', 'None')

**Create Citizenship Column**

In [184]:
#strip citizenship column
cohort_df['citizen'] = cohort_df['citizen'].str.strip()

#convert citizenship column to lowercase
cohort_df['citizen'] = cohort_df['citizen'].str.lower()

#create citizeenship yes, no, status columns
split_column_status(cohort_df, 'citizen', 'south african')

**Create Fullname Columns**

In [185]:
cohort_df['middle_name'] = cohort_df['middle_name'].fillna("")
cohort_df['learner_name'] = cohort_df['learner_name'].str.strip().str.lower() + " " + cohort_df['middle_name'].str.strip().str.lower()
cohort_df['learner_name'] = cohort_df['learner_name'].str.strip().str.title()
cohort_df['learner_fullname'] = cohort_df['learner_name'].str.strip().str.lower() + " " + cohort_df['learner_surname'].str.strip().str.lower()
cohort_df['learner_fullname'] = [re.sub(' +', ' ', fullname) for fullname in cohort_df['learner_fullname']]
cohort_df['learner_fullname'] = cohort_df['learner_fullname'].str.title()
cohort_df['learner_initials'] = [''.join([x[0].upper() for x in fullname.split(' ')]) for fullname in cohort_df['learner_fullname']]


In [186]:
#create full name and initials
#def create_full_name(target_df):
#    target_df['learner_fullname'] = target_df['learner_name'].str.strip().str.lower() + " " + target_df['middle_name'] + " " + target_df['learner_surname'].str.strip().str.lower()
#    target_df['learner_initials'] = [re.sub(' +', ' ', fullname) for fullname in target_df['learner_fullname']]
#    target_df['learner_fullname'] = target_df['learner_fullname'].str.title()
#    target_df['learner_initials'] = [''.join([x[0].upper() for x in fullname.split(' ')]) for fullname in target_df['learner_fullname']]
#    
#    return

#create_full_name(cohort_df)


**Get Specific Students**

In [187]:
cohort_df = subset_dataframe(cohort_df,'Username', ['kmohodi021'])
len(cohort_df)

1

**Create Dates**

In [188]:
#cohort_df['birth_day'] = [date[8:10] for date in cohort_df['birth_date']]
#cohort_df['birth_month'] = [date[5:7] for date in cohort_df['birth_date']]
#cohort_df['birth_year'] = [date[:4] for date in cohort_df['birth_date']]

#cohort_df['birth_date_1'] = [date[8:9] for date in cohort_df['birth_date']]
#cohort_df['birth_date_2'] = [date[9:10] for date in cohort_df['birth_date']]
#cohort_df['birth_date_3'] = [date[5:6] for date in cohort_df['birth_date']]
#cohort_df['birth_date_4'] = [date[6:7] for date in cohort_df['birth_date']]
#cohort_df['birth_date_5'] = [date[:1] for date in cohort_df['birth_date']]
#cohort_df['birth_date_6'] = [date[1:2] for date in cohort_df['birth_date']]
#cohort_df['birth_date_7'] = [date[2:3] for date in cohort_df['birth_date']]
#cohort_df['birth_date_8'] = [date[3:4] for date in cohort_df['birth_date']]
#cohort_df['grade_12'] = "Grade 12"

**Create Residential and Home Address**

In [189]:
cohort_df['home_address'] = cohort_df['home_address_1'] + ", " + cohort_df['home_address_2'] + ", " + cohort_df['home_address_3'] + ", " + cohort_df['home_address_post_code']
cohort_df['residential_address'] = cohort_df['residential_address_1'] + ", " + cohort_df['residential_address_2'] + ", " + cohort_df['residential_address_3'] + ", " + cohort_df['residential_address_post_code']
cohort_df['urban'] = "Urban"


**Create Campus Address Field**

In [190]:
cohort_df['campus_address'] = cohort_df['Campus'].replace({'Durban':'Durban Club Pl, Durban Central, Durban, 4001',
                             'Cape Town':'331 Albert Rd, Woodstock, Cape Town, 7915',
                             'Johannesburg':'84 Albertina Sisulu Rd, Johannesburg, 2000'})

**Verify Columns**

In [199]:
for column in list(filter(lambda x: type(x) == str,tag_ids_df['ABSA_Contract'])):
    if column not in cohort_df.columns:
        print(column)

**Set Learner Email and Template ID**

In [200]:
#cohort_df['learner_email'] = 'mufaro@thoughtquest.co.za'

template_ids = {'MICT':'ab47c0e5-5be4-48c8-bcf1-b4eae380f296', 'Contract':'e94f4098-fe6c-476e-b9d9-12b51aea01bf', 
               'MICT_OLD':'9f5bf080-569e-4f14-8de1-c17cdb51e7c4', 'GPStragies_Contract':'2a99dcb2-af3f-43c9-8ab9-32ca3706e8f0',
               'BBD_MICT':'9389c08b-d25d-43c7-bf24-7d125670c866', 'BBD_Contract':'e4f92cb3-997d-46f2-b164-bfc0083520f9',
               'Entersekt_MICT':'fdcaf1be-bfbf-46f4-a4ee-afb7c626423b', 'Entersekt_Contract':'ba47a206-1c8c-488b-83e0-c2f30d48f3d7',
               'Sybrin_MICT':'074d731e-75e6-4df0-a486-ba4d92be50aa', 'Sybrin_Contract':'7b9ea83b-2c7b-4626-b3f5-d456f0608a98',
               'Outsurance_INSETA':'f4950293-3531-4153-bd1f-0ff20322b26c', 'Outsurance_Contract':'3dc74bbf-e49f-48ae-9ec1-91db58d9ff6c',
               'POPI_INSETA':'768df48e-8b96-48b0-8772-6173e41be9c9', 'Qlink_FASSET':'9ff1f822-0829-49b3-8898-3aa5c781bb54',
               'Qlink_Contract':'a7dfe031-96a2-4811-935f-54d28612a4e3', 'CM_MICT':'7f8f348f-5cd6-4395-a1b4-29e18f4ad5f2',
               'CM_Contract':'a5537079-9726-48a1-bb68-5812efca4371', 'Mediclinic_Contract':'f02a5331-f21e-4ce7-990c-968e00726408',
               'Mediclinic_HWSETA':'1b98d864-d92e-4683-8452-e4d2ad3a585c', 'Sybrin_MICT':'533455b6-235a-4a9b-b1cd-61c01a490cc6',
               'Sybrin_Contract':'a1b797c6-f9c5-47d1-bf04-c2118ca5a171', 'StructureIT_MICT':'69eb896c-bd04-40fe-9598-42fd53b06450',
               'StructureIT_Contract':'6a123619-2449-4c83-8914-d2d4b536f0e0', 'DealX_MICT':'2e86d7bf-f908-4fd1-ab0b-a1a11fbd2ee9',
               'DealX_Contract':'5e14a463-6b06-45fc-ae87-d78aedec7770', 'ABSA_BANKSETA':'9e323179-0e2c-4537-a3d6-7482637cf4ec',
               'ABSA_Contract':'893ab6bc-8373-4401-bfdd-bbc1fd40e7b5'}
cohort_df['template_id'] = template_ids['ABSA_Contract']

target_columns = list(filter(lambda x: type(x) == str,tag_ids_df['ABSA_Contract']))


**Replace All Null Values**

In [193]:
cohort_df = cohort_df.fillna("")

**Send Contracts**

In [201]:
send_bulk_signrequest(cohort_df, target_columns, 'template_id')

Signer: kmohodi021@student.wethinkcode.co.za , Status: 201


In [197]:
cohort_df['learner_fullname']

329    Karabo Faith Mohodi
Name: learner_fullname, dtype: object